In [1]:
import numpy as np
import torch
import torchvision
import torchaudio
import pandas as pd
import matplotlib.pyplot as plt
import einops
import sklearn

Generating data

In [131]:
np.random.seed(45)
num_samples = 40

# Generate data
x1 = np.random.uniform(-1, 1, num_samples)
f_x = 3*x1 + 4
eps = np.random.randn(num_samples)
y = f_x + eps

x = torch.tensor(x1, dtype=torch.float32, requires_grad=False)
y = torch.tensor(y, dtype=torch.float32, requires_grad=False)

Question 1

Finding true gradient on the given data using torch.autograd

In [132]:
#declaring random vector

def val(theta1, theta0):
    theta0 = torch.tensor([float(theta0)], requires_grad=True)
    theta1 = torch.tensor([float(theta1)], requires_grad=True)

    global x,y

    loss = ((y - theta1*x-theta0)**2).mean()
    loss.backward()

    return theta0.grad, theta1.grad

t0grad, t1grad = val(1.0, 1.0)

print(f"True gradient value of gradient of theta 0: {t0grad}")
print(f"True gradient value of gradient of theta 1: {t1grad}")

True gradient value of gradient of theta 0: tensor([-5.6164])
True gradient value of gradient of theta 1: tensor([-0.5630])


In [133]:
theta0 = torch.tensor([1.0], requires_grad=True)
theta1 = torch.tensor([1.0], requires_grad=True)

****Question2****

In this question, we calculate the value of gradient over each point using loss.backward(), which stores the value of gradient in theta1.grad and theta0.grad

Upon storing each value, when we take the average value, we get the value that we obtain using the method true gradient.

In [134]:
gradients_theta_0 = []
gradients_theta_1 = []

for i in range(len(x)):
    #need to set the value to zero, so that the previously stored data doesnot deviate the calculation
    if(i != 0): #excluding the first iterationqq
        theta0.grad.zero_()
        theta1.grad.zero_()

    loss_indi = ((y[i] - theta1*x[i]-theta0)**2)
    loss_indi.backward()

    gradients_theta_0.append(theta0.grad.item())
    gradients_theta_1.append(theta1.grad.item())

avg_SDG_theta0 = np.mean(gradients_theta_0)
avg_SDG_theta1 = np.mean(gradients_theta_1)


In [135]:
#printing all the values

print(f"Grad. desc values for all the points:")

for a in range(len(x)):
    print(f"Theta0 = {round(gradients_theta_0[a],4)}: Theta1 = {round(gradients_theta_1[a],4)}")

Grad. desc values for all the points:
Theta0 = -8.05: Theta1 = -7.8731
Theta0 = -8.6183: Theta1 = -0.854
Theta0 = -3.8722: Theta1 = 1.6926
Theta0 = -5.1751: Theta1 = 4.3752
Theta0 = -4.4476: Theta1 = 0.494
Theta0 = -6.4911: Theta1 = 0.353
Theta0 = -1.5076: Theta1 = 1.3613
Theta0 = -2.4577: Theta1 = 1.6549
Theta0 = -3.3838: Theta1 = 2.5991
Theta0 = -8.1772: Theta1 = -2.0834
Theta0 = -7.1411: Theta1 = -5.087
Theta0 = -5.5181: Theta1 = -1.6566
Theta0 = -8.6144: Theta1 = -8.4545
Theta0 = -5.469: Theta1 = 0.3243
Theta0 = -4.1224: Theta1 = -0.9753
Theta0 = -3.7202: Theta1 = 1.6171
Theta0 = -12.0666: Theta1 = -11.4875
Theta0 = -4.704: Theta1 = -1.6282
Theta0 = -5.0257: Theta1 = 0.5977
Theta0 = -7.8445: Theta1 = 3.2996
Theta0 = -4.8937: Theta1 = -0.0949
Theta0 = -1.0841: Theta1 = 0.8402
Theta0 = -4.3592: Theta1 = 2.3805
Theta0 = -5.8193: Theta1 = 0.2496
Theta0 = -5.6503: Theta1 = 2.907
Theta0 = -1.9239: Theta1 = 0.431
Theta0 = -8.6631: Theta1 = -5.5249
Theta0 = -2.8081: Theta1 = 2.3894
Theta0 

In [136]:
print(avg_SDG_theta0)
print(avg_SDG_theta1)

-5.616434812545776
-0.5629974454641342


As we, can see that upon taking the average of all the gradient values of theta0 and theta1 individually upon each point is equal to the gradient equal to the gradient obtained using true gradient or full batch gradient

***Question 3***

Comparing the epochs required by various method of gradient descent.

The epochs run until the final loss (latest loss) is sufficiently close to the value of the loss at the optimal value of theta vector, that is at the points, that we have obtained using the direct method.

**Implementing full batch gradient descent**

in this the values are updated after each iteration once, that is after each epoch is completed, we update the value of theta vector.

In the following code, Calculating **true gradient**, until convergence value achieved.

In [137]:
#adding one to as the first row.
x_new = torch.stack([torch.ones_like(x), x], dim=1) #creating x vector with 1s and x values.

Finding the true value to find the value of actual theta0 and theta1

In this, we first calculate the optimum value of theta vector using the standard value. Then we calculate the optimum loss, that is plugging in these value inside the loss function. Then iterating thru each loop and breaking when the current loss - optimum loss converge withing the range of epsilon.

In [138]:
#calculating the optimal theta values

xtx_inverse = np.linalg.inv(x_new.T @ x_new)
theta = xtx_inverse @ x_new.T @ y.numpy()
print(theta)
theta0 = theta0_plot = theta[0]
theta1 = theta1_plot =theta[1]

tensor([3.9507, 2.6825])


/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_14043/3607695965.py:4: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  theta = xtx_inverse @ x_new.T @ y.numpy()


In [139]:
print(f"theta0 (intercept): {theta0}")
print(f"theta1 (slope): {theta1}")

#plugging the optimal value of theta0 and theta1 gives us the optimal loss, that is also the minimum loss, as in linear regression we are trying to minimize the loss function, and there is only one minimum point.
loss_optimal = ((y - theta1*x-theta0)**2).mean()
print(f"Loss value: {loss_optimal}")

theta0 (intercept): 3.9507062435150146
theta1 (slope): 2.682469367980957
Loss value: 0.5957541465759277


Setting hyperparameters as follows

In [140]:
#setting some hyperparamaters

alpha = 0.0005
convergence_threshold = 0.001 #given in question

Calculating **True Gradient**

In [141]:
final_list = [] #this is to store all the lists that the function returns for different and random values of theta0 and theta1.
average_iter = []

In [142]:
#following part has to be changed, as to initialize it randomly
#Finding the average number of iterations using three epoch values


#setting the first value of theta0 and theta1 as the following.
#for the first case setting a proper value (which is random for the system, but for the sake of the question, it is set to 2.5, for comparison in plotting)

theta0_init = 2.5
theta1_init = 2.5
theta0 = torch.tensor([theta0_init], requires_grad=True) 
theta1 = torch.tensor([theta1_init], requires_grad=True)

Defining the function, which returns three lists -> theta0, theta1 and loss after each epoch.

Inside the function, there is a while loop, which is set true, and it breaks only when, the current loss function, is epsilon close to the optimal theta value.

In [143]:

def true_gradient(theta0, theta1):
    theta0 = torch.tensor([float(theta0)], requires_grad=True)
    theta1 = torch.tensor([float(theta1)], requires_grad=True)

    global x,y, alpha, convergence_threshold, theta0_plot, theta1_plot, loss_optimal
    val_theta0 = [theta0.item()]
    val_theta1 = [theta1.item()]
    loss_values = [loss_optimal.item()]

    epoch_vanilla = 0
    epoch_f = 200000 #maximum number of epochs, just to contraint if there is no convergence.

    while epoch_vanilla < epoch_f:
        
        loss = ((y - theta1 * x - theta0) ** 2).mean()
        loss.backward()

        with torch.no_grad():
        
            theta0 -= alpha * theta0.grad
            theta1 -= alpha * theta1.grad
        
        val_theta0.append(theta0.detach().item())
        val_theta1.append(theta1.detach().item())
        loss_values.append(loss.item())

        distance = torch.sqrt((theta0 - theta0_plot) ** 2 + (theta1 - theta1_plot) ** 2)
        if (abs(distance) < convergence_threshold):
            break
        epoch_vanilla += 1

        
        theta0.grad.zero_()
        theta1.grad.zero_()


    print(f"Converged after {epoch_vanilla} epochs")
    print(f"Final theta0: {theta0.item()}")
    print(f"Final theta1: {theta1.item()}")
    return val_theta0, val_theta1, loss_values


In [144]:
#giving theta0 and theta1 as the input to the function, and then the three lists obtained are stored in a list called finallist.

val_theta0, val_theta1, loss_values = true_gradient(theta0_init, theta1_init)
final_list.append([val_theta0, val_theta1, loss_values])
average_iter.append(len(val_theta0))

Converged after 19500 epochs
Final theta0: 3.9505045413970947
Final theta1: 2.681490182876587


In [145]:

# def true_gradient(theta0, theta1):
#     theta0 = torch.tensor([float(theta0)], requires_grad=True)
#     theta1 = torch.tensor([float(theta1)], requires_grad=True)

#     global x,y, alpha, convergence_threshold
#     val_theta0 = [theta0.item()]
#     val_theta1 = [theta1.item()]
#     loss_values = [loss_optimal.item()]

#     epoch_vanilla = 0
#     epoch_f = 200000 #maximum number of epochs, just to contraint if there is no convergence.

#     while epoch_vanilla < epoch_f:
        
#         loss = ((y - theta1 * x - theta0) ** 2).mean()
#         loss.backward()

#         with torch.no_grad():
        
#             theta0 -= alpha * theta0.grad
#             theta1 -= alpha * theta1.grad
        
#         val_theta0.append(theta0.detach().item())
#         val_theta1.append(theta1.detach().item())
#         loss_values.append(loss.item())

        
#         theta0.grad.zero_()
#         theta1.grad.zero_()

#         if (abs(loss_optimal - loss) < convergence_threshold):
#             break
#         epoch_vanilla += 1

#     print(f"Converged after {epoch_vanilla} epochs")
#     print(f"Final theta0: {theta0.item()}")
#     print(f"Final theta1: {theta1.item()}")
#     return val_theta0, val_theta1, loss_values


In [146]:
#appending the data of first random value of theta0 and theta1 as a CSV file for plotting and visualization
data_vanilla = pd.DataFrame({
    'val_theta0': val_theta0,
    'val_theta1': val_theta1,
    'loss_values': loss_values
})
data_vanilla.to_csv('processed_data.csv', index=False)


Running the trial for second random value

In [147]:
#random values
theta0_rand_values = np.random.uniform(0, 5, 10)
theta0_rand_values = np.random.uniform(0, 5, 10)

In [148]:
for a in range(9):
    theta0 = torch.tensor([theta0_rand_values[a]], requires_grad=True)
    theta1 = torch.tensor([theta0_rand_values[a]], requires_grad=True)
    val_theta0, val_theta1, loss_values = true_gradient(theta0_rand_values[a], theta0_rand_values[a])
    final_list.append([val_theta0, val_theta1, loss_values])
    average_iter.append(len(val_theta0))

Converged after 26352 epochs
Final theta0: 3.9505045413970947
Final theta1: 2.681490182876587
Converged after 21294 epochs
Final theta0: 3.9505045413970947
Final theta1: 2.681490182876587
Converged after 25698 epochs
Final theta0: 3.9509084224700928
Final theta1: 2.683448553085327
Converged after 24938 epochs
Final theta0: 3.9505045413970947
Final theta1: 2.681490182876587
Converged after 20643 epochs
Final theta0: 3.9505045413970947
Final theta1: 2.681490182876587
Converged after 26128 epochs
Final theta0: 3.9505045413970947
Final theta1: 2.681490182876587
Converged after 25519 epochs
Final theta0: 3.9509084224700928
Final theta1: 2.683448553085327
Converged after 22424 epochs
Final theta0: 3.9505045413970947
Final theta1: 2.681490182876587
Converged after 26439 epochs
Final theta0: 3.9505045413970947
Final theta1: 2.681490182876587


Printing the average number of iterations required for the convergence of the loss function.
Here the number of iterations equal to the number of epochs required for the convergence of the loss function.

In [149]:
print(f"Average number of epochs/iteration: {np.mean(average_iter)}")

Average number of epochs/iteration: 23895.5


SGD method for the epoch calculation

In [150]:
alpha_sgd = 0.0001
convergence_threshold = 0.001
epoch_f_sgd = 50000
sgd_avg_iteration = []

In [151]:
theta0_sgd = torch.tensor([2.5], requires_grad=True)
theta1_sgd = torch.tensor([2.5], requires_grad=True)

In [152]:
def sgd(theta0_sgd, theta1_sgd,x, y, alpha_sgd, convergence_threshold, loss_optimal, epoch_f_sgd):

    val_theta0_sgd = [theta0_sgd.item()]
    val_theta1_sgd = [theta1_sgd.item()]
    loss_values_sgd = [loss_optimal.item()] 

    epoch_sgd = 0

    flag = True

    while epoch_sgd < epoch_f_sgd and flag:
        indices = torch.randperm(x.size(0))
        x = x[indices]
        y = y[indices]
        for i in range(len(x)):
            prediction = theta1_sgd * x[i] + theta0_sgd
            loss_sgd = (y[i] - prediction) ** 2
            loss_sgd.backward()

            with torch.no_grad():
                theta0_sgd -= alpha_sgd * theta0_sgd.grad
                theta1_sgd -= alpha_sgd * theta1_sgd.grad

            theta0_sgd.grad.zero_()
            theta1_sgd.grad.zero_()

        total_loss_sgd = ((y - (theta1_sgd * x + theta0_sgd)) ** 2).mean()

        distance = torch.sqrt((theta0_sgd - theta0_plot) ** 2 + (theta1_sgd - theta1_plot) ** 2)
        if (abs(distance) < convergence_threshold):
            break

        val_theta0_sgd.append(theta0_sgd.detach().item())
        val_theta1_sgd.append(theta1_sgd.detach().item())
        loss_values_sgd.append(total_loss_sgd.item())

        epoch_sgd += 1

    print(f"Converged after {epoch_sgd} epochs")
    print(f"Final theta0: {theta0_sgd.item()}")
    print(f"Final theta1: {theta1_sgd.item()}")
    print(f"Final loss: {loss_values_sgd[-1]}")
    return val_theta0_sgd, val_theta1_sgd, loss_values_sgd

In [153]:
# def sgd(theta0_sgd, theta1_sgd,x, y, alpha_sgd, convergence_threshold, loss_optimal, epoch_f_sgd):

#     val_theta0_sgd = [theta0_sgd.item()]
#     val_theta1_sgd = [theta1_sgd.item()]
#     loss_values_sgd = [loss_optimal.item()] 

#     epoch_sgd = 0

#     flag = True

#     while epoch_sgd < epoch_f_sgd and flag:
        
#         for i in range(len(x)):
#             prediction = theta1_sgd * x[i] + theta0_sgd
#             loss_sgd = (y[i] - prediction) ** 2
#             loss_sgd.backward()

#             with torch.no_grad():
#                 theta0_sgd -= alpha_sgd * theta0_sgd.grad
#                 theta1_sgd -= alpha_sgd * theta1_sgd.grad

#             theta0_sgd.grad.zero_()
#             theta1_sgd.grad.zero_()

#         total_loss_sgd = ((y - (theta1_sgd * x + theta0_sgd)) ** 2).mean()

#         if (abs(loss_optimal - loss_sgd) < convergence_threshold):
#             break

#         val_theta0_sgd.append(theta0_sgd.detach().item())
#         val_theta1_sgd.append(theta1_sgd.detach().item())
#         loss_values_sgd.append(total_loss_sgd.item())

#         epoch_sgd += 1

#     print(f"Converged after {epoch_sgd} epochs")
#     print(f"Final theta0: {theta0_sgd.item()}")
#     print(f"Final theta1: {theta1_sgd.item()}")
#     print(f"Final loss: {loss_values_sgd[-1]}")
#     return val_theta0_sgd, val_theta1_sgd, loss_values_sgd

In [154]:
val_theta0_sgd, val_theta1_sgd, loss_values_sgd = sgd(theta0_sgd, theta1_sgd, x, y, alpha_sgd, convergence_threshold, loss_optimal, epoch_f_sgd)
sgd_avg_iteration.append(len(val_theta0_sgd))

Converged after 2344 epochs
Final theta0: 3.95086932182312
Final theta1: 2.681483745574951
Final loss: 0.5957545638084412


In [155]:
data_sgd = pd.DataFrame({
    'val_theta0_sgd': val_theta0_sgd,
    'val_theta1_sgd': val_theta1_sgd,
    'loss_values_sgd': loss_values_sgd
})
data_sgd.to_csv('processed_data_sgd.csv', index=False)


In [156]:
#doing the same for random values of theta0 and theta1
for a in range(len(theta0_rand_values)):
    theta0_sgd = torch.tensor([theta0_rand_values[a]], requires_grad=True)
    theta1_sgd = torch.tensor([theta0_rand_values[a]], requires_grad=True)
    val_theta0_sgd, val_theta1_sgd, loss_values_sgd = sgd(theta0_sgd, theta1_sgd, x, y, alpha_sgd, convergence_threshold, loss_optimal, epoch_f_sgd)
    sgd_avg_iteration.append(len(val_theta0_sgd))

Converged after 3234 epochs
Final theta0: 3.95084036882625
Final theta1: 2.6814802044671175
Final loss: 0.595754528851232
Converged after 2602 epochs
Final theta0: 3.9508404363507594
Final theta1: 2.6814807672804513
Final loss: 0.5957545285147067
Converged after 3236 epochs
Final theta0: 3.95107447144578
Final theta1: 2.683398119514469
Final loss: 0.5957545296218004
Converged after 3057 epochs
Final theta0: 3.95084028529229
Final theta1: 2.681479520376889
Final loss: 0.5957545292598979
Converged after 2520 epochs
Final theta0: 3.950840233432425
Final theta1: 2.6814791129855107
Final loss: 0.5957545295024788
Converged after 3206 epochs
Final theta0: 3.950840383441109
Final theta1: 2.681480324225685
Final loss: 0.595754528779716
Converged after 3214 epochs
Final theta0: 3.951074389099826
Final theta1: 2.6833974448842186
Final loss: 0.5957545292185148
Converged after 2743 epochs
Final theta0: 3.9508403605647255
Final theta1: 2.6814801401559403
Final loss: 0.5957545288894588
Converged afte

Printing the average number of epochs required for convergence of the loss function.
Here the average number of iterations required is equal to the number of epochs x number of elements in the dataset


In [157]:
print(f"Average number of epochs for SGD: {np.mean(sgd_avg_iteration)}")
print(f"Average number of iterations for SGD: {np.mean(sgd_avg_iteration)*len(x)}")

Average number of epochs for SGD: 2965.5454545454545
Average number of iterations for SGD: 118621.81818181818


Changing the algorithm

Gradient Descent using Mini-Batch method.

In [158]:

def mini_batch_gradient_descent(x, y, theta0_mini_batch, theta1_mini_batch, alpha, batch_size, convergence_threshold, epoch_f_mini_batch):

    val_theta0_mini_batch = [theta0_mini_batch.item()]
    val_theta1_mini_batch = [theta1_mini_batch.item()]
    loss_values_mini_batch = [loss_optimal.item()]
    global alpha_mini_batch, theta0_plot, theta1_plot
    epoch_mini_batch = 0
    prev_loss = 0

    while (epoch_mini_batch < epoch_f_mini_batch):
        indices = torch.randperm(x.size(0))
        x = x[indices]
        y = y[indices]

        for i in range(0, x.size()[0], batch_size):
            batch_x = x[i:i + batch_size]
            batch_y = y[i: i + batch_size]

            loss_mini_batch = ((batch_y - theta1_mini_batch * batch_x - theta0_mini_batch) ** 2).mean()
            loss_mini_batch.backward()

            with torch.no_grad():
                theta0_mini_batch -= alpha_mini_batch * theta0_mini_batch.grad
                theta1_mini_batch -= alpha_mini_batch * theta1_mini_batch.grad

            theta0_mini_batch.grad.zero_()
            theta1_mini_batch.grad.zero_()

        total_loss_mini_batch = ((y - (theta1_mini_batch * x + theta0_mini_batch)) ** 2).mean()

        val_theta0_mini_batch.append(theta0_mini_batch.detach().item())
        val_theta1_mini_batch.append(theta1_mini_batch.detach().item())
        loss_values_mini_batch.append(total_loss_mini_batch.item())

        distance = torch.sqrt((theta0_mini_batch - theta0_plot) ** 2 + (theta1_mini_batch - theta1_plot) ** 2)
        if (abs(distance) < convergence_threshold):
            break

        epoch_mini_batch += 1

    print(f"Converged after {epoch_mini_batch} epochs")
    print(f"Final theta0: {theta0_mini_batch.item()}")
    print(f"Final theta1: {theta1_mini_batch.item()}")
    return val_theta0_mini_batch, val_theta1_mini_batch, loss_values_mini_batch

In [159]:

# def mini_batch_gradient_descent(x, y, theta0_mini_batch, theta1_mini_batch, alpha, batch_size, convergence_threshold, epoch_f_mini_batch):

#     val_theta0_mini_batch = [theta0_mini_batch.item()]
#     val_theta1_mini_batch = [theta1_mini_batch.item()]
#     loss_values_mini_batch = [loss_optimal.item()]

#     epoch_mini_batch = 0
#     prev_loss = 0

#     while (epoch_mini_batch < epoch_f_mini_batch):
#         indices = torch.randperm(x.size(0))
#         x = x[indices]
#         y = y[indices]

#         for i in range(0, x.size()[0], batch_size):
#             batch_x = x[i:i + batch_size]
#             batch_y = y[i: i + batch_size]

#             loss_mini_batch = ((batch_y - theta1_mini_batch * batch_x - theta0_mini_batch) ** 2).mean()
#             loss_mini_batch.backward()

#             with torch.no_grad():
#                 theta0_mini_batch -= alpha_mini_batch * theta0_mini_batch.grad
#                 theta1_mini_batch -= alpha_mini_batch * theta1_mini_batch.grad

#             theta0_mini_batch.grad.zero_()
#             theta1_mini_batch.grad.zero_()

#         total_loss_mini_batch = ((y - (theta1_mini_batch * x + theta0_mini_batch)) ** 2).mean()

#         val_theta0_mini_batch.append(theta0_mini_batch.detach().item())
#         val_theta1_mini_batch.append(theta1_mini_batch.detach().item())
#         loss_values_mini_batch.append(total_loss_mini_batch.item())

#         if abs(total_loss_mini_batch - loss_optimal) < convergence_threshold:
#             break

#         epoch_mini_batch += 1

#     print(f"Converged after {epoch_mini_batch} epochs")
#     print(f"Final theta0: {theta0_mini_batch.item()}")
#     print(f"Final theta1: {theta1_mini_batch.item()}")
#     return val_theta0_mini_batch, val_theta1_mini_batch, loss_values_mini_batch

In [160]:
batch_size = 10
alpha_mini_batch = 0.0001
convergence_threshold = 0.001
epoch_f_mini_batch = 100000

#for the first iteration, setting the values of theta0 and theta1 as 2.5, for comparison in plotting
theta0_mini_batch = torch.tensor([2.5], requires_grad=True)
theta1_mini_batch = torch.tensor([3.0], requires_grad=True)

In [161]:
mini_batch_iterations = []

In [162]:
val_theta0_mini_batch, val_theta1_mini_batch, loss_values_mini_batch = mini_batch_gradient_descent(x, y, theta0_mini_batch, theta1_mini_batch, alpha, batch_size, convergence_threshold, epoch_f_mini_batch)
mini_batch_iterations.append(len(val_theta0_mini_batch))

Converged after 20249 epochs
Final theta0: 3.9508249759674072
Final theta1: 2.683461904525757


In [163]:
data_vanilla = pd.DataFrame({
    'val_theta0': val_theta0_mini_batch,
    'val_theta1': val_theta1_mini_batch,
    'loss_values_mini_batch': loss_values_mini_batch
})
data_vanilla.to_csv('processed_data_mini_batch.csv', index=False)

In [164]:
mini_batch_iterations = []
for a in range(len(theta0_rand_values)-1):
    #batch_size = 10
    #alpha_mini_batch = 0.0005
    #convergence_threshold = 0.001
    #epoch_f_mini_batch = 100000
    theta0_mini_batch = torch.tensor([theta0_rand_values[a]], requires_grad=True)
    theta1_mini_batch = torch.tensor([theta0_rand_values[a]], requires_grad=True)
    val_theta0_mini_batch, val_theta1_mini_batch, loss_values_mini_batch = mini_batch_gradient_descent(x, y, theta0_mini_batch, theta1_mini_batch, alpha, batch_size, convergence_threshold, epoch_f_mini_batch)
    mini_batch_iterations.append(len(val_theta0_mini_batch))
    

Converged after 32609 epochs
Final theta0: 3.9505859613299554
Final theta1: 2.681476686604201
Converged after 26286 epochs
Final theta0: 3.9505857303611154
Final theta1: 2.681476807112101
Converged after 31794 epochs
Final theta0: 3.950827542856923
Final theta1: 2.6834617400342577
Converged after 30841 epochs
Final theta0: 3.950585450348337
Final theta1: 2.681476773923651
Converged after 25472 epochs
Final theta0: 3.9505862743390616
Final theta1: 2.681476823984855
Converged after 32328 epochs
Final theta0: 3.9505845313165064
Final theta1: 2.6814768828014066
Converged after 31571 epochs
Final theta0: 3.9508280452336035
Final theta1: 2.683461840467764
Converged after 27697 epochs
Final theta0: 3.9505857184137483
Final theta1: 2.6814766648648805
Converged after 32717 epochs
Final theta0: 3.9505857835964076
Final theta1: 2.6814766736215616


Printing the number of average epochs required tested over different vector of theta values.
also printing the average number of iterations required for convergence of the loss function.

In [165]:
print(f"Average number of iterations for mini-batch gradient descent: {np.mean(mini_batch_iterations)}")
print(f"Standard deviation of number of iterations for mini-batch gradient descent: {np.mean(mini_batch_iterations)*10}")

Average number of iterations for mini-batch gradient descent: 30148.11111111111
Standard deviation of number of iterations for mini-batch gradient descent: 301481.1111111111


Maximum number of epochs or iterations (average) are being required for the mini batch gradient descent, followed by full batch gradient descent and then the stochastic gradient descent.

The reason for this can be the following:

1) This model updated the values least number of times, as compared to the other two, as it updates only once per epoch, that is less frequency of updates.
2) These updates can be stated as more accurate, as it takes the average of all the points, and then updates the value of theta vector.
3) This results in a slow convergence of the loss function, as compared to the other two methods.

Visualization using Contour plots.

Aim is to miminize the difference between the current loss which is based on the current (updated) value of theta, and the optimal loss.

In this the z axis represent the difference, while x and y axis represent theta_0 ad theta_1 respectively.

Done in File Final_plot_Q3

****Question 4****

In this, in the updation rule, there is an addition of **additional term**.

This term is the product of **constant value momentum and the previous value of theta vector**.


**Why do we need Momentum based gradient descent ?**

1. In case there is a very gentle slope, then there wont be any significant change in the value of theta vector, and the vector gets stuck in that region.
2. This is because the gradient in these regions were small.

**How do we modify the algorithm.**

1. As we go in a particular direction, our momentum in that particular direction has some impact. Similarly, if we are descending thru a slope of gradient descent, our momentum in that particular direction increases, and hence its better to include that momentum term along with the gradient term.

2. This can be done, by incorporating some term from the previous update, to increase the movement in that particular direction.

let v be the theta vector.

$$
\mathbf{v} = \begin{bmatrix} \theta_0 \\ \theta_1 \end{bmatrix}
$$

$$
\mathbf{v}_{t+1} = \mathbf{v}_t - \text{update}_t
$$

where 

$$
\text{update}_t = \gamma \cdot \text{update}_{t-1} + \alpha \cdot \nabla \mathbf{v}_t
$$


Momentum based implementation

In [166]:

def true_gradient_momentum(theta0, theta1, x, y, alpha, convergence_threshold, loss_optimal, epoch_f, gamma):


    v_theta0 = torch.zeros_like(theta0)
    v_theta1 = torch.zeros_like(theta1)

    val_theta0_momentum = [theta0.item()]
    val_theta1_momentum = [theta1.item()]
    loss_values_momentum = [loss_optimal.item()]

    #As in the visualization part, we also need to plot the vector of grad and momentum, we require some more lists
    grad_theta0_list = []
    grad_theta1_list = []
    momentum_theta0_list = []
    momentum_theta1_list = []

    epoch_vanilla = 0

    while epoch_vanilla < epoch_f:
        loss = ((y - theta1 * x - theta0) ** 2).mean()
        loss.backward()

        with torch.no_grad():

            grad_theta0_list.append(theta0.grad.item())
            grad_theta1_list.append(theta1.grad.item())

            v_theta0 = gamma * v_theta0 + alpha * theta0.grad
            v_theta1 = gamma * v_theta1 + alpha * theta1.grad

            momentum_theta0_list.append(v_theta0.item())
            momentum_theta1_list.append(v_theta1.item())
            
            theta0 -= v_theta0
            theta1 -= v_theta1
        
        val_theta0_momentum.append(theta0.detach().item())
        val_theta1_momentum.append(theta1.detach().item())
        loss_values_momentum.append(loss.item())

        theta0.grad.zero_()
        theta1.grad.zero_()

        distance = torch.sqrt((theta0 - theta0_plot) ** 2 + (theta1 - theta1_plot) ** 2)
        if (abs(distance) < convergence_threshold):
            break

        epoch_vanilla += 1

    print(f"Converged after {epoch_vanilla} epochs")
    print(f"Final theta0: {theta0.item()}")
    print(f"Final theta1: {theta1.item()}")
    return val_theta0_momentum, val_theta1_momentum, loss_values_momentum, grad_theta0_list, grad_theta1_list, momentum_theta0_list, momentum_theta1_list

Random Value - 1

In [167]:
# aveg_iteration_truegrad_momentum = []
# alpha = 0.01
# gamma = 0.9
# convergence_threshold = 0.001
# epoch_f = 20000

In [168]:
aveg_iteration_truegrad_momentum = []
alpha = 0.01
gamma = 0.9
convergence_threshold = 0.001
epoch_f = 20000

theta0 = torch.tensor([2.5], requires_grad=True)
theta1 = torch.tensor([2.5], requires_grad=True)
val_theta0_momentum, val_theta1_momentum, loss_values_momentum, grad_theta0_list, grad_theta1_list, momentum_theta0_list, momentum_theta1_list = true_gradient_momentum(theta0, theta1, x, y, alpha, convergence_threshold, loss_optimal, epoch_f, gamma)
aveg_iteration_truegrad_momentum.append(len(val_theta0_momentum)-2)

data_momentum = pd.DataFrame({
    'val_theta0': val_theta0_momentum,
    'val_theta1': val_theta1_momentum,
    'loss_values': loss_values_momentum,
    'grad_theta0': grad_theta0_list + [0.0],
    'grad_theta1': grad_theta1_list + [0.0],
    'momentum_theta0': momentum_theta0_list + [0.0],
    'momentum_theta1': momentum_theta1_list + [0.0]
})

data_momentum.to_csv('true_gradient_momentum.csv', index=False)

for a in range(len(theta0_rand_values)-1):
    theta0_momentum = torch.tensor([float(theta0_rand_values[a])], requires_grad=True)  # reinitialize for each iteration
    theta1_momentum = torch.tensor([float(theta0_rand_values[a])], requires_grad=True)  # reinitialize for each iteration
    
    val_theta0_momentum, val_theta1_momentum, loss_values_momentum, grad_theta0_list, grad_theta1_list, momentum_theta0_list, momentum_theta1_list = true_gradient_momentum(theta0_momentum, theta1_momentum, x, y, alpha, convergence_threshold, loss_optimal, epoch_f, gamma)
    
    aveg_iteration_truegrad_momentum.append(len(val_theta0_momentum)-2)

Converged after 125 epochs
Final theta0: 3.951430559158325
Final theta1: 2.6819002628326416
Converged after 148 epochs
Final theta0: 3.950218439102173
Final theta1: 2.6832096576690674
Converged after 126 epochs
Final theta0: 3.9511966705322266
Final theta1: 2.681659698486328
Converged after 134 epochs
Final theta0: 3.9512546062469482
Final theta1: 2.6833016872406006
Converged after 145 epochs
Final theta0: 3.9498283863067627
Final theta1: 2.6829066276550293
Converged after 126 epochs
Final theta0: 3.9511818885803223
Final theta1: 2.6817967891693115
Converged after 147 epochs
Final theta0: 3.9500367641448975
Final theta1: 2.6831421852111816
Converged after 134 epochs
Final theta0: 3.951188087463379
Final theta1: 2.6832616329193115
Converged after 127 epochs
Final theta0: 3.950914144515991
Final theta1: 2.681494951248169
Converged after 148 epochs
Final theta0: 3.9502105712890625
Final theta1: 2.6832289695739746


Random values using the list of theta values which are randomly generated.

In [169]:
print("average number of epochs for true gradient momentum: ", np.mean(aveg_iteration_truegrad_momentum))
print("average number of iterations for true gradient momentum: ", np.mean(aveg_iteration_truegrad_momentum))


average number of epochs for true gradient momentum:  136.0
average number of iterations for true gradient momentum:  136.0


Applying momentum based gradient descent for SGD

In [170]:

def sgd_momentum(theta0_sgd, theta1_sgd, x, y, alpha_sgd, convergence_threshold, loss_optimal, gamma_sgd):

    global theta0_plot, theta1_plot

    v_theta0_sgd = torch.zeros_like(theta0_sgd)
    v_theta1_sgd = torch.zeros_like(theta1_sgd)

    val_theta0_sgd = [theta0_sgd.item()]
    val_theta1_sgd = [theta1_sgd.item()]
    loss_values_sgd = [loss_optimal.item()]

    grad_theta0_sgd_list = []
    grad_theta1_sgd_list = []
    momentum_theta0_sgd_list = []
    momentum_theta1_sgd_list = []

    epoch_sgd = 0
    epoch_f_sgd = 20000 #limiting in case the values do not converge
    flag = True

    while epoch_sgd < epoch_f_sgd and flag:
        indices = torch.randperm(x.size(0))
        x = x[indices]
        y = y[indices]

        grad_theta0_epoch = []
        grad_theta1_epoch = []
        momentum_theta0_epoch = []
        momentum_theta1_epoch = []
        for i in range(len(x)):
            prediction = theta1_sgd * x[i] + theta0_sgd
            loss_sgd = (y[i] - prediction) ** 2
            loss_sgd.backward()

            with torch.no_grad():

                grad_theta0_epoch.append(theta0_sgd.grad.item())
                grad_theta1_epoch.append(theta1_sgd.grad.item())

                v_theta0_sgd = gamma_sgd * v_theta0_sgd + alpha_sgd * theta0_sgd.grad
                v_theta1_sgd = gamma_sgd * v_theta1_sgd + alpha_sgd * theta1_sgd.grad

                momentum_theta0_epoch.append(v_theta0_sgd.item())
                momentum_theta1_epoch.append(v_theta1_sgd.item())
                
                theta0_sgd -= v_theta0_sgd
                theta1_sgd -= v_theta1_sgd

                theta0_sgd.grad.zero_()
                theta1_sgd.grad.zero_()

        total_loss_sgd = ((y - (theta1_sgd * x + theta0_sgd)) ** 2).mean()

        distance = torch.sqrt((theta0_sgd - theta0_plot) ** 2 + (theta1_sgd - theta1_plot) ** 2)
        if (abs(distance) < convergence_threshold):
            break

        grad_theta0_sgd_list.append(np.mean(grad_theta0_epoch))
        grad_theta1_sgd_list.append(np.mean(grad_theta1_epoch))
        momentum_theta0_sgd_list.append(np.mean(momentum_theta0_epoch))
        momentum_theta1_sgd_list.append(np.mean(momentum_theta1_epoch))

        val_theta0_sgd.append(theta0_sgd.detach().item())
        val_theta1_sgd.append(theta1_sgd.detach().item())
        loss_values_sgd.append(total_loss_sgd.item())

        epoch_sgd += 1

    print(f"Converged after {epoch_sgd} epochs")
    print(f"Final theta0: {theta0_sgd.item()}")
    print(f"Final theta1: {theta1_sgd.item()}")
    print(f"Final loss: {loss_values_sgd[-1]}")
    return val_theta0_sgd, val_theta1_sgd, loss_values_sgd, grad_theta0_sgd_list, grad_theta1_sgd_list, momentum_theta0_sgd_list, momentum_theta1_sgd_list

Random Value - 1

In [171]:
aveg_iteration_sgd_momentum = []
alpha_sgd = 0.0001
gamma_sgd = 0.5
convergence_threshold = 0.001

In [172]:
theta0_sgd = torch.tensor([2.5], requires_grad=True)
theta1_sgd = torch.tensor([2.5], requires_grad=True)
val_theta0_sgd, val_theta1_sgd, loss_values_sgd, grad_theta0_sgd_list, grad_theta1_sgd_list, momentum_theta0_sgd_list, momentum_theta1_sgd_list = sgd_momentum(theta0_sgd, theta1_sgd, x, y, alpha_sgd, convergence_threshold, loss_optimal, gamma_sgd)
aveg_iteration_sgd_momentum.append(len(val_theta0_sgd)-1)

Converged after 1161 epochs
Final theta0: 3.9508306980133057
Final theta1: 2.681478261947632
Final loss: 0.5957545638084412


In [173]:
data_sgd = pd.DataFrame({
    'val_theta0_sgd': val_theta0_sgd,
    'val_theta1_sgd': val_theta1_sgd,
    'loss_values_sgd': loss_values_sgd,
    'grad_theta0_sgd': grad_theta0_sgd_list + [0.0],
    'grad_theta1_sgd': grad_theta1_sgd_list + [0.0],
    'momentum_theta0_sgd': momentum_theta0_sgd_list + [0.0],
    'momentum_theta1_sgd': momentum_theta1_sgd_list + [0.0]
})

data_sgd.to_csv('true_gradient_sgd_momentum.csv', index=False)

In [174]:
for a in range(len(theta0_rand_values)-1):
    theta0_sgd = torch.tensor([theta0_rand_values[a]], requires_grad=True)
    theta1_sgd = torch.tensor([theta0_rand_values[a]], requires_grad=True)
    val_theta0_sgd, val_theta1_sgd, loss_values_sgd, grad_theta0_sgd_list, grad_theta1_sgd_list, momentum_theta0_sgd_list, momentum_theta1_sgd_list = sgd_momentum(theta0_sgd, theta1_sgd, x, y, alpha_sgd, convergence_threshold, loss_optimal, gamma_sgd)
    aveg_iteration_sgd_momentum.append(len(val_theta0_sgd)-1)

Converged after 1585 epochs
Final theta0: 3.950842545236206
Final theta1: 2.6814807146534267
Final loss: 0.5957545315252417
Converged after 1269 epochs
Final theta0: 3.9508425713343676
Final theta1: 2.681480944417092
Final loss: 0.5957545313864545
Converged after 1664 epochs
Final theta0: 3.951076126371595
Final theta1: 2.683395388871288
Final loss: 0.5957545298932395
Converged after 1497 epochs
Final theta0: 3.9508427989483974
Final theta1: 2.681482794785603
Final loss: 0.5957545302777555
Converged after 1228 epochs
Final theta0: 3.9508423292382124
Final theta1: 2.681478973366601
Final loss: 0.5957545325699848
Converged after 1571 epochs
Final theta0: 3.9508425597208707
Final theta1: 2.6814808334250784
Final loss: 0.5957545314539385
Converged after 1652 epochs
Final theta0: 3.9510764715538933
Final theta1: 2.683398218344595
Final loss: 0.5957545315896098
Converged after 1340 epochs
Final theta0: 3.950842860280933
Final theta1: 2.6814833025958578
Final loss: 0.595754529973355
Converged

In [175]:
print(f"Average number of epochs for SGD with momentum: {np.mean(aveg_iteration_sgd_momentum)}")
print(f"Average number of iterations for SGD with momentum: {np.mean(aveg_iteration_sgd_momentum)*40}")

Average number of epochs for SGD with momentum: 1455.8
Average number of iterations for SGD with momentum: 58232.0


Mini Batch Gradient with momentum updation

In [176]:
batch_size = 10
alpha_mini_batch = 0.0001
gamma_mini_batch = 0.9
convergence_threshold = 0.001

In [177]:

def minibatch_momentum(theta0_mini_batch, theta1_mini_batch , x, y, alpha_mini_batch, convergence_threshold, loss_optimal, gamma_mini_batch):
    
    v_theta0_mini_batch = torch.zeros_like(theta0_mini_batch)
    v_theta1_mini_batch = torch.zeros_like(theta1_mini_batch)

    val_theta0_mini_batch = [theta0_mini_batch.item()]
    val_theta1_mini_batch = [theta1_mini_batch.item()]
    loss_values_mini_batch = [loss_optimal.item()]

    grad_theta0_minibatch = []
    grad_theta1_minibatch = []
    momentum_theta0_minibatch= []
    momentum_theta1_minibatch = []

    epoch_mini_batch = 0
    epoch_f_mini_batch = 20000

    while epoch_mini_batch < epoch_f_mini_batch:
        indices = torch.randperm(x.size(0))
        x = x[indices]
        y = y[indices]

        grad_theta0_batch = []
        grad_theta1_batch = []
        momentum_theta0_batch = []
        momentum_theta1_batch = []
        num_batches = 0

        for i in range(0, x.size()[0], batch_size):
            batch_x = x[i: i + batch_size]
            batch_y = y[i: i + batch_size]

            loss_mini_batch = ((batch_y - theta1_mini_batch * batch_x - theta0_mini_batch) ** 2).mean()
            loss_mini_batch.backward()

            with torch.no_grad():
                v_theta0_mini_batch = gamma_mini_batch * v_theta0_mini_batch + alpha_mini_batch * theta0_mini_batch.grad
                v_theta1_mini_batch = gamma_mini_batch * v_theta1_mini_batch + alpha_mini_batch * theta1_mini_batch.grad
                
                theta0_mini_batch -= v_theta0_mini_batch
                theta1_mini_batch -= v_theta1_mini_batch

                grad_theta0_batch.append(theta0_mini_batch.grad.item())
                grad_theta1_batch.append(theta1_mini_batch.grad.item())
                momentum_theta0_batch.append(v_theta0_mini_batch.item())
                momentum_theta1_batch.append(v_theta1_mini_batch.item())

            theta0_mini_batch.grad.zero_()
            theta1_mini_batch.grad.zero_()
            num_batches += 1

        total_loss_mini_batch = ((y - (theta1_mini_batch * x + theta0_mini_batch)) ** 2).mean()

        val_theta0_mini_batch.append(theta0_mini_batch.detach().item())
        val_theta1_mini_batch.append(theta1_mini_batch.detach().item())
        loss_values_mini_batch.append(total_loss_mini_batch.item())

        grad_theta0_minibatch.append(np.mean(grad_theta0_batch))
        grad_theta1_minibatch.append(np.mean(grad_theta1_batch))
        momentum_theta0_minibatch.append(np.mean(momentum_theta0_batch))
        momentum_theta1_minibatch.append(np.mean(momentum_theta1_batch))

        distance = torch.sqrt((theta0_mini_batch - theta0_plot) ** 2 + (theta1_mini_batch - theta1_plot) ** 2)
        if (abs(distance) < convergence_threshold):
            break

        epoch_mini_batch += 1

    print(f"Converged after {epoch_mini_batch} epochs")
    print(f"Final theta0: {theta0_mini_batch.item()}")
    print(f"Final theta1: {theta1_mini_batch.item()}")
    return val_theta0_mini_batch, val_theta1_mini_batch, loss_values_mini_batch, grad_theta0_minibatch, grad_theta1_minibatch, momentum_theta0_minibatch, momentum_theta1_minibatch


In [178]:
mini_batch_iterations = []

In [179]:
theta0_mini_batch = torch.tensor([2.5], requires_grad=True)
theta1_mini_batch = torch.tensor([2.5], requires_grad=True)
val_theta0_mini_batch, val_theta1_mini_batch, loss_values_mini_batch, grad_theta0_minibatch, grad_theta1_minibatch, momentum_theta0_minibatch, momentum_theta1_minibatch = minibatch_momentum(theta0_mini_batch, theta1_mini_batch, x, y, alpha_mini_batch, convergence_threshold, loss_optimal, gamma_mini_batch)
mini_batch_iterations.append(len(val_theta0_mini_batch)-2)


Converged after 2395 epochs
Final theta0: 3.9505698680877686
Final theta1: 2.681480646133423


In [180]:
data_mini_batch = pd.DataFrame({
    'val_theta0_mini_batch': val_theta0_mini_batch,
    'val_theta1_mini_batch': val_theta1_mini_batch,
    'loss_values_mini_batch': loss_values_mini_batch,
    'grad_theta0_mini_batch': grad_theta0_minibatch + [0.0],
    'grad_theta1_mini_batch': grad_theta1_minibatch + [0.0],
    'momentum_theta0_mini_batch': momentum_theta0_minibatch + [0.0],
    'momentum_theta1_mini_batch': momentum_theta1_minibatch + [0.0]
})

data_mini_batch.to_csv('true_gradient_mini_batch_momentum.csv', index=False)

In [181]:
for a in range(len(theta0_rand_values)-1):
    theta0_mini_batch = torch.tensor([theta0_rand_values[a]], requires_grad=True)
    theta1_mini_batch = torch.tensor([theta0_rand_values[a]], requires_grad=True)
    val_theta0_mini_batch, val_theta1_mini_batch, loss_values_mini_batch, grad_theta0_minibatch, grad_theta1_minibatch, momentum_theta0_minibatch, momentum_theta1_minibatch = minibatch_momentum(theta0_mini_batch, theta1_mini_batch, x, y, alpha_mini_batch, convergence_threshold, loss_optimal, gamma_mini_batch)
    mini_batch_iterations.append(len(val_theta0_mini_batch)-2)

Converged after 3245 epochs
Final theta0: 3.9505735402316686
Final theta1: 2.681478358954157
Converged after 2617 epochs
Final theta0: 3.95057372222035
Final theta1: 2.681479857134945
Converged after 3161 epochs
Final theta0: 3.9508154511854663
Final theta1: 2.6834622423575163
Converged after 3070 epochs
Final theta0: 3.9505737560224827
Final theta1: 2.6814801287831025
Converged after 2536 epochs
Final theta0: 3.9505736906708555
Final theta1: 2.6814796032875114
Converged after 3218 epochs
Final theta0: 3.950573802625103
Final theta1: 2.681480510825374
Converged after 3139 epochs
Final theta0: 3.950815393420549
Final theta1: 2.6834617686331916
Converged after 2757 epochs
Final theta0: 3.9505735931290515
Final theta1: 2.681478794737748
Converged after 3256 epochs
Final theta0: 3.950573607111548
Final theta1: 2.6814789074234273


In [182]:
print(f"Average number of epochs for Mini Batch with momentum: {np.mean(mini_batch_iterations)}")
print(f"Average number of iterations for Mini Batch with momentum: {np.mean(mini_batch_iterations)*10}")

Average number of epochs for Mini Batch with momentum: 2939.4
Average number of iterations for Mini Batch with momentum: 29394.0


The average number of steps taken are least by true gradient by full batch gradient descent, followed by SGD with momnetum and then mini batch gradient descent with momentum.

The epochs taken by mini batch are high as compared to the epochs required by other two